In [64]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import json
import time
import os

from src.neuron_heads import head_attribution_over_all_data
from src.datahandlers import ActivatingDataset
from src.utils import tuple_str_to_tuple
from src.neuron_explain import generate_classify_prompt_dict

/Users/clementneo/projects/Neuron_groups/neuron_groups/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# filename = "2023-10-13_13-51-37_gpt2-small_test"
# filename = "2023-10-13_14-02-06_gpt2-medium_test"
# filename = "2023-10-13_14-17-42_gpt2-large_test"
filename = "2023-10-14_18-35-10_gpt2-large_mid_test"

with open(f'../experiment_data/4_head_attributions/{filename}.json') as f:
    head_attributions = json.load(f)

trimmed_texts_filename = head_attributions['prior_filename']
with open(f'../experiment_data/3_trimmed_texts/{trimmed_texts_filename}.json') as f:
    trimmed_texts = json.load(f)

max_activating_filename = trimmed_texts['prior_filename']
with open(f'../experiment_data/2_max_activating_texts/{max_activating_filename}.json') as f:
    max_activating = json.load(f)

neuron_filename = max_activating['prior_filename']
with open(f'../experiment_data/1_next_token_neurons/{neuron_filename}.json') as f:
    neurons_data = json.load(f)

neurons = [tuple_str_to_tuple(neuron_str) for neuron_str in head_attributions['head_attributions'].keys()]

neuron_to_token = {tuple_str_to_tuple(neuron_str): token_data['token'] for neuron_str, token_data in neurons_data['neurons'].items()}

# Parameters
parameters = head_attributions['parameters']
model_name = parameters['model_name']

In [4]:
# Load explanations

import json
# Load the jsonl
with open('../experiment_data/5_head_explanations/2023-10-14_16-00-09_gpt2-large_mid_prompts_mid_results.jsonl', 'r') as f:
# with open('../experiment_data/5_head_explanations/2023-10-12_14-43-12_gpt2-small_prompts_results.jsonl', 'r') as f:
# with open('../experiment_data/5_head_explanations/2023-10-12_14-23-51_gpt2-medium_prompts_results.jsonl', 'r') as f:
# with open('../experiment_data/5_head_explanations/2023-10-12_14-41-45_gpt2-large_prompts_results.jsonl', 'r') as f:
    lines = f.readlines()
    lines = [json.loads(line) for line in lines]

x = lines[0]
# print(x[0]['messages'])
exp_prompt_to_exp = {x[0]['messages'][0]['content']: x[1]['choices'][0]['message']['content'] for x in lines}

with open('../experiment_data/5_head_explanations/2023-10-14_16-00-09_gpt2-large_mid_prompts_dict.json') as f:
# with open('../experiment_data/5_head_explanations/2023-10-12_14-43-12_gpt2-small_prompts_dict.json') as f:
# with open('../experiment_data/5_head_explanations/2023-10-12_14-23-51_gpt2-medium_prompts_dict.json') as f:
# with open('../experiment_data/5_head_explanations/2023-10-12_14-41-45_gpt2-large_prompts_dict.json') as f:
    nh_to_exp_prompt = json.load(f)

nh_to_exp_prompt = {tuple_str_to_tuple(k): v for k, v in nh_to_exp_prompt.items()}

nh_to_exp = {nh: exp_prompt_to_exp[exp_prompt] for nh, exp_prompt in nh_to_exp_prompt.items()}

In [5]:
import pickle

with open('../experiment_data/text_list_dict_test.pkl', 'rb') as f:
    dataset_text_dict = pickle.load(f)

dataset_text_list = [x['text'] for x in dataset_text_dict]
dataset = dataset_text_dict

In [6]:
neuron_to_prompts = {tuple_str_to_tuple(neuron): 
                     list(head_attribution.keys()) 
                     for neuron, head_attribution in head_attributions['head_attributions'].items()}

In [7]:
gpt_4_prompts_dict = generate_classify_prompt_dict(
    nh_to_exp = nh_to_exp,
    neuron_to_prompts=neuron_to_prompts,
    neuron_to_token=neuron_to_token,
    max_per_nh=20
)

# gpt_4_prompts_dict_10 = generate_classify_prompt_dict(
#     nh_to_exp = nh_to_exp,
#     neuron_to_prompts=neuron_to_prompts,
#     neuron_to_token=neuron_to_token,
#     max_per_nh=10
# )

# gpt_4_prompts_dict_20 = generate_classify_prompt_dict(
#     nh_to_exp = nh_to_exp,
#     neuron_to_prompts=neuron_to_prompts,
#     neuron_to_token=neuron_to_token,
#     max_per_nh=20
# )

In [8]:
gpt_4_prompts = []
for nh, exp in gpt_4_prompts_dict.items():
    gpt_4_prompts += list(exp.keys())

len(gpt_4_prompts)

6528

In [10]:
jobs = [
            {"model":"gpt-4",
            "messages":[{"role": "user", "content": gpt_4_prompt}],
            "max_tokens":1, 
        } for gpt_4_prompt in gpt_4_prompts]

filepath = f"../experiment_data/6_explanation_eval/{filename}_prompts_20.jsonl"
if os.path.isfile(filepath):
    raise Exception("File already exists!")

with open(filepath, "w") as f:
    for job in jobs:
        json_string = json.dumps(job)
        f.write(json_string + "\n")

In [12]:
output = {
    'parameters': parameters,
    'prior_filename': filename,
    'classify_filepath': filepath,
    'classify_prompts': {str(k):v for k,v in gpt_4_prompts_dict.items()}
}

# Save json to ../experiment_data/2_max_activating_texts
timestamp = time.strftime("%Y-%m-%d_%H-%M-%S", time.localtime(int(time.time())))
new_filename = f"{timestamp}_{model_name}_mid_test_20.json"

with open(f'../experiment_data/6_explanation_eval/{new_filename}', 'w') as f:
    json.dump(output, f)

In [93]:
# Open jsonl
# filepath = f"../experiment_data/6_explanation_eval/2023-10-13_13-51-37_gpt2-small_test_prompts_results.jsonl"
# filepath = f"../experiment_data/6_explanation_eval/2023-10-13_14-02-06_gpt2-medium_test_prompts_results.jsonl"
# filepath = f"../experiment_data/6_explanation_eval/2023-10-13_14-17-42_gpt2-large_test_prompts_results.jsonl"
#small 11.6, med 15.3,  large 36.96, 

# Explanations
# filepath = f"../experiment_data/5_head_explanations/2023-10-12_14-43-12_gpt2-small_prompts_results.jsonl"
filepath = "../experiment_data/5_head_explanations/2023-10-12_14-23-51_gpt2-medium_prompts_results.jsonl"
# filepath = "../experiment_data/5_head_explanations/2023-10-12_14-41-45_gpt2-large_prompts_results.jsonl"
# small 5.3, med 7.5,  large 18

print(5.3+7.5+12.8)
print(11.6+15.3+36.96)

with open(filepath, "r") as f:
    results = [json.loads(line) for line in f.readlines()]

usages = [results[n][1]['usage'] for n in range(len(results))]
prompt_token_cost = 0.03/1000
completion_token_cost = 0.06/1000
prompt_cost = sum([x['prompt_tokens'] for x in usages]) * prompt_token_cost
completion_cost = sum([x['completion_tokens'] for x in usages]) * completion_token_cost
total_cost = prompt_cost + completion_cost

print(f"Total cost: {total_cost}")

25.6
63.86
Total cost: 7.513859999999999


In [15]:
len(gpt_4_prompts)

6528

In [16]:
prompt_token_cost = 0.03/1000

import tiktoken
encoding = tiktoken.encoding_for_model('gpt-4')

# Count the number of tokens in `gpt_4_prompts`
prompt_tokens = 0
for gpt_4_prompt in gpt_4_prompts:
    prompt_tokens += len(encoding.encode(gpt_4_prompt))



print(prompt_tokens)
print(prompt_tokens * prompt_token_cost)


1183566
35.50698


In [ ]:
# head_attribution_dict = head_attributions['head_attributions']
# nh_to_pos_examples = {}

# NUM_TEST_EXAMPLES = 10
# for nh, data in gpt_4_prompts_dict_10.items():
#     neuron_str, head = str((nh[0], nh[1])), nh[2]
#     num_pos_examples = 0
#     for i, trunc_prompt_with_token in enumerate(data.values()):
#         trunc_prompt = trunc_prompt_with_token[:-len(neuron_to_token[(nh[0], nh[1])])]
#         if head in head_attribution_dict[neuron_str][trunc_prompt]:
#             num_pos_examples += 1
#     nh_to_pos_examples[nh] = num_pos_examples/NUM_TEST_EXAMPLES

# from matplotlib import pyplot as plt
# plt.hist(nh_to_pos_examples.values(), bins=8)
# # plt.show()

# head_attribution_dict = head_attributions['head_attributions']
# nh_to_pos_examples = {}
# NUM_TEST_EXAMPLES = 20

# for nh, data in gpt_4_prompts_dict_20.items():
#     neuron_str, head = str((nh[0], nh[1])), nh[2]
#     num_pos_examples = 0
#     for i, trunc_prompt_with_token in enumerate(data.values()):
#         if i >= NUM_TEST_EXAMPLES: break
#         trunc_prompt = trunc_prompt_with_token[:-len(neuron_to_token[(nh[0], nh[1])])]
#         if head in head_attribution_dict[neuron_str][trunc_prompt]:
#             num_pos_examples += 1
#     nh_to_pos_examples[nh] = num_pos_examples/NUM_TEST_EXAMPLES

# from matplotlib import pyplot as plt
# plt.hist(nh_to_pos_examples.values(), alpha=0.5, bins=8)
# # Put this to 50% transparency
# # plt.show()

In [234]:
# Read test_prompts and _test_prompts_20
# with open("../experiment_data/6_explanation_eval/2023-10-13_14-17-42_gpt2-large_test_prompts.jsonl", 'rb') as f:
# with open("../experiment_data/6_explanation_eval/2023-10-13_14-02-06_gpt2-medium_test_prompts.jsonl", 'rb') as f:
with open("../experiment_data/6_explanation_eval/2023-10-13_13-51-37_gpt2-small_test_prompts.jsonl", "rb") as f:
    test_prompts = [json.loads(line) for line in f.readlines()]

# with open("../experiment_data/6_explanation_eval/2023-10-13_14-17-42_gpt2-large_test_prompts_20.jsonl", 'rb') as f:
# with open("../experiment_data/6_explanation_eval/2023-10-13_14-02-06_gpt2-medium_test_prompts_20.jsonl", 'rb') as f:
with open("../experiment_data/6_explanation_eval/2023-10-13_13-51-37_gpt2-small_test_prompts_20.jsonl", "rb") as f:
    test_prompts_20 = [json.loads(line) for line in f.readlines()]

In [235]:
print(len(test_prompts))
print(len(test_prompts_20))

# Check how many elements in test_prompts_20 that are not in test_prompts

test_prompts_20_prompts = [x['messages'][0]['content'] for x in test_prompts_20]
test_prompts_prompts = [x['messages'][0]['content'] for x in test_prompts]

new_gpt_4_prompts = [x for x in test_prompts_20_prompts if x not in test_prompts_prompts]

print(len(new_gpt_4_prompts))

2070
4096
2026


In [236]:
jobs = [
            {"model":"gpt-4",
            "messages":[{"role": "user", "content": gpt_4_prompt}],
            "max_tokens":1, 
        } for gpt_4_prompt in new_gpt_4_prompts]

# filename = "2023-10-13_14-17-42_gpt2-large_test"
# filename = "2023-10-13_14-02-06_gpt2-medium_test"
filename = "2023-10-13_13-51-37_gpt2-small_test"

filepath = f"../experiment_data/6_explanation_eval/{filename}_prompts_10.jsonl"
# if os.path.isfile(filepath):
#     raise Exception("File already exists!")

# with open(filepath, "w") as f:
#     for job in jobs:
#         json_string = json.dumps(job)
#         f.write(json_string + "\n")

In [237]:
output = {
    'parameters': parameters,
    'prior_filename': filename,
    'classify_filepath': filepath,
    'classify_prompts': {str(k):v for k,v in gpt_4_prompts_dict.items()}
}

# Save json to ../experiment_data/2_max_activating_texts
timestamp = time.strftime("%Y-%m-%d_%H-%M-%S", time.localtime(int(time.time())))
new_filename = f"{filename}_10.json"

with open(f'../experiment_data/6_explanation_eval/{new_filename}', 'w') as f:
    json.dump(output, f)